In [ ]:
import altair as alt
import geopandas as gpd
import pandas as pd
import polars as pl

from great_tables import GT, _data_color, loc, md, nanoplot_options, style

from update_vars import BUS_SERVICE_GCS

In [ ]:
def category_wrangling(
    df: pd.DataFrame, 
    col: str = "category", 
    sort_key: list = ["on_shn", "parallel", "other", "shn_subtotal", "total"]
) -> pd.DataFrame:
    """
    Custom sort order for categorical variable
    https://stackoverflow.com/questions/23482668/sorting-by-a-custom-list-in-pandas
    """
    category_values = {
        "on_shn": "On SHN", 
        "parallel": "Intersects SHN",
        "other": "Other",
        "shn_subtotal": "On or Intersects SHN",
        "total": "Total"
    }
    
    df = df.sort_values(
        col, key=lambda c: c.map(lambda e: sort_key.index(e))
    ) 
    
    df = df.assign(
        category = df.category.map(category_values)
    )
    
    return df

def get_hex(color_name: str) -> str:
    """
    Since some of the color names don't pull the hex code, 
    we'll grab it here.
    https://github.com/posit-dev/great-tables/blob/main/great_tables/_data_color/constants.py
    """
    return _data_color.constants.COLOR_NAME_TO_HEX[color_name]

In [ ]:
current_quarter = "2024-Q2"

operator_df = pd.read_parquet(
    f"{BUS_SERVICE_GCS}"
    "quarterly_metrics/operator_time_series.parquet",
    filters = [[("year_quarter", "==", current_quarter)]]
).pipe(category_wrangling)

district_df = pd.read_parquet(
    f"{BUS_SERVICE_GCS}"
    "quarterly_metrics/district_time_series.parquet",
    filters = [[("year_quarter", "==", current_quarter)]]
).pipe(category_wrangling)

statewide_df = pd.read_parquet(
    f"{BUS_SERVICE_GCS}"
    "quarterly_metrics/statewide_time_series.parquet",
    filters = [[("year_quarter", "==", current_quarter)]]
).pipe(category_wrangling)

In [ ]:
def shared_nano_options(
    point_stroke_color: str,
    line_stroke_color: str,
    point_fill_color: str,
    area_fill_color: str
):
    nano_options = nanoplot_options(
        data_point_radius=6,
        data_point_stroke_color=get_hex(point_stroke_color),
        data_point_fill_color=get_hex(point_fill_color),
        data_point_stroke_width=3,
        data_line_type="curved",
        data_line_stroke_color=get_hex(line_stroke_color),
        data_line_stroke_width=8,
        data_area_fill_color=get_hex(area_fill_color),
        #vertical_guide_stroke_color=None,
        show_y_axis_guide=True,
        #show_vertical_guides=False,
        interactive_data_values = True,
        #reference_line_color=get_hex("salmon1"),
        show_reference_line=False
    )
    
    return nano_options

In [ ]:
def plot_table(df: pd.DataFrame): 
    
    MIN_SPEED, MAX_SPEED = df.speed_mph.min(), df.speed_mph.max()
    MIN_SERVICE, MAX_SERVICE = df.service_hours_per_route.min(), df.service_hours_per_route.max()
    
    table = (
        GT(pl.from_pandas(df))
        .fmt_nanoplot(
            columns="speed_mph_ts",
            plot_type="line",
            expand_y=[round(MIN_SPEED, 0), round(MAX_SPEED, 0)],
            options=shared_nano_options(
                point_stroke_color = "black",
                line_stroke_color = "green",
                point_fill_color = "white",
                area_fill_color = "seagreen2")
        ).fmt_nanoplot(
            columns="service_hours_per_route_ts",
            plot_type="line",
            expand_y=[round(MIN_SERVICE, 0), round(MAX_SERVICE, 0)],
            options=shared_nano_options(
                point_stroke_color = "black", 
                line_stroke_color = "steelblue1",
                point_fill_color = "white",
                area_fill_color = "lightskyblue2", 
            )
        ).fmt_number(
            columns = [
                "daily_service_hours", 
                "service_hours_per_route", 
                "speed_mph"], decimals=1
        ).fmt_integer(
            columns = ["daily_routes", "daily_vp_routes"]
         ).cols_label(
            category = "Category",
            daily_service_hours = "Daily Service Hours",
            service_hours_per_route = "Service Hours per Route",
            speed_mph = "Average Speed",
            daily_routes = "# Routes",
            daily_vp_routes = "# Routes",
            service_hours_per_route_ts = "Service Hours per Route (time-series)",
            speed_mph_ts = "Speed (time-series)",
         ).tab_header(
             title = "Service Hours and Speed",
             subtitle = f"{current_quarter}"
         ).tab_spanner(
            label="Service", 
            columns=["daily_service_hours", 
                     "service_hours_per_route", "daily_routes", 
                     "service_hours_per_route_ts"]
        ).tab_spanner(
            label="Speed (mph)",
            columns = ["speed_mph", "daily_vp_routes", "speed_mph_ts"]
        ).tab_options(
            container_width = "100%",
            table_background_color="white",
            table_body_hlines_style="none",
            table_body_vlines_style="none",
            heading_background_color="white",
            column_labels_background_color="white",
            row_group_background_color="white",
            stub_background_color="white",
            source_notes_background_color="white",
            table_font_size="14px",
            heading_align="center"
        ).cols_hide(
            ["year_quarter", "service_hours", "n_routes", 
            "n_dates", "n_vp_routes"]
        ).sub_missing(
            columns = ["speed_mph", "speed_mph_ts", "daily_vp_routes"],
            missing_text = ""
        ).tab_style(
            style=style.text(weight="bold"),
            locations=loc.body(rows=pl.col("category") == "Total")
        ).tab_style(
            style=style.text(
                weight="normal", style="italic", color=get_hex("gray20")),
            locations=loc.body(
                rows=pl.col("category") == "On or Intersects SHN"),
        ).cols_align(align="center")
         .cols_align(align="left", columns="category")
        
    )
    
    return table

## Statewide Metrics

In [ ]:
plot_table(statewide_df)

## District Breakdown

In [ ]:
def district_table_specs(table, one_district, one_quarter):
    table2 = (table
              .cols_hide("caltrans_district")
              .tab_header(
                 title = f"District {one_district}",
                 subtitle = f"Service Hours and Speed  {one_quarter}")
             )
    
    return table2
    

for i in sorted(district_df.caltrans_district.unique()):
    table = plot_table(
        district_df[district_df.caltrans_district==i])
    
    table = district_table_specs(table, i, current_quarter)

    display(table)

## Operator Breakdown

In [ ]:
plot_table(operator_df)